In [42]:
# from translate import Translator
from langdetect import detect, detect_langs
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environ

In [43]:
use_date_str = False

In [44]:
conn = psycopg2.connect(
host = environ['DB_HOST'],
port = environ['DB_PORT'],
user = environ['DB_USER'],
password = environ['DB_PASSWORD'],
database = environ['DB_NAME'])

In [45]:
limit = 500

if use_date_str == False:
    query = r"""
        SELECT a.id, r.raw_html, a.language 
        FROM articles a
        INNER JOIN raw_html r
            ON a.id = r.article_id
        WHERE a.article_type = 'blog'
        AND a.parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
else:
    query = r"""
        SELECT a.id, r.raw_html, a.posted_date_str, a.language 
        FROM articles a
        INNER JOIN raw_html r
            ON a.id = r.article_id
        WHERE a.posted_date_str IS NOT NULL
        AND a.parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_17858/1328800568.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,raw_html,language
0,1474,"<html lang=""en"" class=""DeviceDetect--isDesktop...",en
1,2773,"<html lang=""kk"" dir=""ltr"" class="" js""><head>\n...",uk
2,2838,"<html lang=""en"" data-rh=""lang""><head><title>Ho...",en
3,3391,"<html lang=""en"" data-rh=""lang""><head><title>Ac...",en
4,3392,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en
...,...,...,...
87,13014,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en
88,13168,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en
89,13305,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en
90,13306,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en


In [46]:
# reg = r"(?<=>)(.*?)(?=<\/)"
reg = r"<p>(.*?)<\/p>"
reg_date = r'<h6 class="posted-date">(.*?)</h6>'

def extractParagraphs (row):
    try:
        return ''.join(re.findall(reg, row['raw_html']))
    except:
        return ''

In [47]:
# body['raw'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg, x)))
body['raw'] = body.apply(extractParagraphs, axis = 1)

if use_date_str == False:
    body['date'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg_date, x)).strip().lower().replace(r'[\s\n]+', ' '))
else:
    body['date'] = body['posted_date_str']

In [48]:
def verifyLanguage (row):
    lang = row['language']
    try:
        if lang == 'en':
            return detect(row['raw'])
        else:
            return lang
    except:
        return lang

In [49]:
body['verified_language'] = body.apply(verifyLanguage, axis = 1)
body[body['language'] != body['verified_language']]

,id,raw_html,language,raw,date,verified_language


In [50]:
def translate (row):
    lang = row['verified_language']
    # translator = Translator(from_lang = lang, to_lang = 'en')
    # return translator.translate(row['date'].strip().upper())
    if lang != 'en':
        try:
            return ts.translate_text(row['date'], from_language = lang, to_language = 'en')
        except:
            return row['date']
    else:
        return row['date']

In [51]:
def parseDate (row):
    try:
        return parser.parse(row['date_translation'])
    except:
        return None

In [52]:
body['date_translation'] = body.apply(translate, axis = 1)
body['datetime'] = body.apply(parseDate, axis = 1)
body

,id,raw_html,language,raw,date,verified_language,date_translation,datetime
0,1474,"<html lang=""en"" class=""DeviceDetect--isDesktop...",en,,,en,,NaT
1,2773,"<html lang=""kk"" dir=""ltr"" class="" js""><head>\n...",uk,Қазакстандағы БҰҰДБ-ң гендерлік мәселелер бойы...,2023 ж. 27 april,uk,2023 f. 27 April,NaT
2,2838,"<html lang=""en"" data-rh=""lang""><head><title>Ho...",en,,,en,,NaT
3,3391,"<html lang=""en"" data-rh=""lang""><head><title>Ac...",en,,,en,,NaT
4,3392,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,This might be because:United NationsDevelopmen...,,en,,NaT
...,...,...,...,...,...,...,...,...
87,13014,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,Achieving sustainability in the agricultural s...,10 de octubre de 2018,en,10 de octubre de 2018,NaT
88,13168,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,United NationsDevelopment Programme,,en,,NaT
89,13305,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,Photo: UNDP.The first birds awaken the winter ...,4 de agosto de 2021,en,4 de agosto de 2021,NaT
90,13306,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,"Photo: Rodrigo Razquín/MovésOn April 8, 2021, ...",11 de marzo de 2022,en,11 de marzo de 2022,NaT


In [53]:
body[body['datetime'].isna()]

,id,raw_html,language,raw,date,verified_language,date_translation,datetime
0,1474,"<html lang=""en"" class=""DeviceDetect--isDesktop...",en,,,en,,NaT
1,2773,"<html lang=""kk"" dir=""ltr"" class="" js""><head>\n...",uk,Қазакстандағы БҰҰДБ-ң гендерлік мәселелер бойы...,2023 ж. 27 april,uk,2023 f. 27 April,NaT
2,2838,"<html lang=""en"" data-rh=""lang""><head><title>Ho...",en,,,en,,NaT
3,3391,"<html lang=""en"" data-rh=""lang""><head><title>Ac...",en,,,en,,NaT
4,3392,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,This might be because:United NationsDevelopmen...,,en,,NaT
...,...,...,...,...,...,...,...,...
87,13014,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,Achieving sustainability in the agricultural s...,10 de octubre de 2018,en,10 de octubre de 2018,NaT
88,13168,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,United NationsDevelopment Programme,,en,,NaT
89,13305,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,Photo: UNDP.The first birds awaken the winter ...,4 de agosto de 2021,en,4 de agosto de 2021,NaT
90,13306,"<html dir=""ltr"" class="" js"" data-once=""media-l...",en,"Photo: Rodrigo Razquín/MovésOn April 8, 2021, ...",11 de marzo de 2022,en,11 de marzo de 2022,NaT


In [54]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [55]:
cur = conn.cursor()

for i, r in body[body['datetime'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET parsed_date = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['datetime'], r['id']))

conn.commit()
cur.close()

In [56]:
## CLOSE THE CONNECTION TO THE DB
conn.close()